In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")

#emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa' , return_all_scores= True)  # https://stackoverflow.com/questions/71557275/emotion-detection-in-text-using-emoroberta

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_a

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/IDSIA Biomedical Texts/tweets_to_annotate.xlsx")
df

,status_id,user_id,text,category
0,1245998640499322880,2677225992,Episode of #CoronaVirus panic. Man took his gl...,PP
1,1246166884531219968,2526719574,Don’t let this #coronavirus bullshit steer you...,NP
2,1246189954163760896,917105229467979776,When you are having a panic attack and one of ...,PO
3,1246206351145144064,31396876,#BlackWidow is back on the schedule! Now if I ...,PP
4,1250061443816448000,2329229755,The COVId 19 panic is so real that if someone ...,PO
...,...,...,...,...
211,1250572544215642112,1172938416125808896,Having a panic attack while grocery shopping i...,PP
212,1248497721775780096,74114924,"Since my anxiety around #coronavirus subsided,...",PO
213,1248656868941103104,1224352945833160960,"Personally, I don't have a problem with #StayA...",UN
214,1248716546656144896,20393302,Have reverted back to having panic attacks dur...,PP


In [ ]:
text_list = list(df['text'])

In [ ]:
len(text_list)

216

In [ ]:
import tensorflow as tf

In [ ]:
emo_label_prob = []

for item in text_list:
  encoded_input = tokenizer(item, truncation=True, padding=True, return_tensors="tf")
  output = model(encoded_input["input_ids"], attention_mask=encoded_input["attention_mask"])

  emotions = output.logits[0]#.numpy() # these are logits, not probabilities, this needs torch.sigmoid() or torch.nn.functional.softmax() (Ref- https://stackoverflow.com/questions/46416984/how-to-convert-logits-to-probability-in-binary-classification-in-tensorflow)
  emotions_prob = tf.math.softmax(emotions).numpy() # we use tensorflow softmax here to convert logits to probabilities

  emotion_labels = model.config.id2label.values()
  emotion_probabilities = dict(zip(emotion_labels, emotions_prob))

  emo_label_prob.append(emotion_probabilities)

In [ ]:
emo_label_prob[0]

[[{'label': 'admiration', 'score': 5.9294070524629205e-05},
  {'label': 'amusement', 'score': 0.0002665968204382807},
  {'label': 'anger', 'score': 0.0011603842722252011},
  {'label': 'annoyance', 'score': 0.0231387447565794},
  {'label': 'approval', 'score': 0.0030824088025838137},
  {'label': 'caring', 'score': 0.0005665787612088025},
  {'label': 'confusion', 'score': 0.000617641897406429},
  {'label': 'curiosity', 'score': 0.0003229469293728471},
  {'label': 'desire', 'score': 8.282100316137075e-05},
  {'label': 'disappointment', 'score': 0.024537911638617516},
  {'label': 'disapproval', 'score': 0.0013354686088860035},
  {'label': 'disgust', 'score': 0.0025460009928792715},
  {'label': 'embarrassment', 'score': 0.7947162389755249},
  {'label': 'excitement', 'score': 0.00022600092052016407},
  {'label': 'fear', 'score': 0.0338142067193985},
  {'label': 'gratitude', 'score': 0.0005093544605188072},
  {'label': 'grief', 'score': 2.312976175744552e-05},
  {'label': 'joy', 'score': 5.20

In [ ]:
df['emo_label_prob'] = emo_label_prob
df

,status_id,user_id,text,category,emo_label_prob
0,1245998640499322880,2677225992,Episode of #CoronaVirus panic. Man took his gl...,PP,"{'admiration': 5.9294165e-05, 'amusement': 0.0..."
1,1246166884531219968,2526719574,Don’t let this #coronavirus bullshit steer you...,NP,"{'admiration': 0.00016007239, 'amusement': 6.3..."
2,1246189954163760896,917105229467979776,When you are having a panic attack and one of ...,PO,"{'admiration': 0.00023742238, 'amusement': 0.0..."
3,1246206351145144064,31396876,#BlackWidow is back on the schedule! Now if I ...,PP,"{'admiration': 0.000534939, 'amusement': 0.000..."
4,1250061443816448000,2329229755,The COVId 19 panic is so real that if someone ...,PO,"{'admiration': 0.0003127861, 'amusement': 0.00..."
...,...,...,...,...,...
211,1250572544215642112,1172938416125808896,Having a panic attack while grocery shopping i...,PP,"{'admiration': 0.00023126583, 'amusement': 0.0..."
212,1248497721775780096,74114924,"Since my anxiety around #coronavirus subsided,...",PO,"{'admiration': 9.971763e-05, 'amusement': 0.00..."
213,1248656868941103104,1224352945833160960,"Personally, I don't have a problem with #StayA...",UN,"{'admiration': 0.00012573857, 'amusement': 1.6..."
214,1248716546656144896,20393302,Have reverted back to having panic attacks dur...,PP,"{'admiration': 6.874266e-05, 'amusement': 0.00..."


In [ ]:
emo_df = df['emo_label_prob'].apply(pd.Series)

In [ ]:
len(emo_df.columns)

28

In [ ]:
# concatenate this with the original to get everything in one dataframe
df = pd.concat([df, emo_df], axis=1)
df.head()

,status_id,user_id,text,category,emo_label_prob,admiration,amusement,anger,annoyance,approval,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,1245998640499322880,2677225992,Episode of #CoronaVirus panic. Man took his gl...,PP,"{'admiration': 5.9294165e-05, 'amusement': 0.0...",0.000059,0.000267,0.001160,0.023139,0.003082,...,0.000275,0.082093,0.000126,0.001421,0.002678,0.001155,0.012150,0.010215,0.000147,0.002682
1,1246166884531219968,2526719574,Don’t let this #coronavirus bullshit steer you...,NP,"{'admiration': 0.00016007239, 'amusement': 6.3...",0.000160,0.000063,0.001678,0.000607,0.001919,...,0.000232,0.004161,0.001674,0.000115,0.000235,0.005173,0.000314,0.000201,0.000157,0.003289
2,1246189954163760896,917105229467979776,When you are having a panic attack and one of ...,PO,"{'admiration': 0.00023742238, 'amusement': 0.0...",0.000237,0.000132,0.000366,0.001924,0.056736,...,0.000341,0.351478,0.000905,0.000067,0.004076,0.001257,0.000207,0.001354,0.000754,0.034777
3,1246206351145144064,31396876,#BlackWidow is back on the schedule! Now if I ...,PP,"{'admiration': 0.000534939, 'amusement': 0.000...",0.000535,0.000172,0.000043,0.000637,0.706703,...,0.000056,0.001282,0.056095,0.000529,0.027748,0.003876,0.000125,0.000080,0.000223,0.189041
4,1250061443816448000,2329229755,The COVId 19 panic is so real that if someone ...,PO,"{'admiration': 0.0003127861, 'amusement': 0.00...",0.000313,0.000563,0.000073,0.001056,0.066868,...,0.000069,0.007173,0.004606,0.000059,0.134053,0.000484,0.000094,0.001890,0.003335,0.761554


In [ ]:
# df.to_csv("Tweets Emotions.csv", index=False)